In [1]:
import pandas as pd
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
from dataops import *
import tushare as ts
# 显示所有行（取消行的截断）
pd.set_option('display.max_rows', None)

# 显示所有列（取消列的截断）
pd.set_option('display.max_columns', None)

# 显示每列的所有内容（避免内容过长被截断）
pd.set_option('display.max_colwidth', None)
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

#step1: 数据加载
start_date = "2020-01-01"
end_date = "2020-03-01"
def getdata(code = "sh.600519", start_date = start_date, end_date = end_date):
    df = get_stock_data_from_baostock(code = code, start_date = start_date, end_date = end_date, login=False)
    df.index = pd.to_datetime(df["date"])
    df["openinterest"] = 0
    df = df[["open", "high", "low", "close", "volume", "openinterest"]]
    print(df)
    print(df["low"].dtype)

    columns_to_convert = ["open", "high", "low", "close", "volume", "openinterest"]  # 替换为你的列名

    # 3. 逐个转换为数值类型
    for col in columns_to_convert:
        # errors='coerce'会将无法转换的值设为NaN
        # downcast='float'或'integer'可以优化存储
        df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
    print(df["low"].dtype)

    # df["low"] = df["low"].astype(float)
    # print(df["low"].dtype)

    return df


def getdata2(code = "sh.600519", start_date = start_date, end_date = end_date):
    df = ts.pro_bar(ts_code = code, start_date = start_date, end_date = end_date)
    print(df)
    return df


stock_df = getdata2(code = "sh.600519", start_date = start_date, end_date = end_date)

#加载并读取数据(bt固定写法)
#dataname:数据df; fromdate:回测开始时间; todate:回测结束时间
fromdate = datetime(2020, 1, 1)  #2020年1月1日，date格式
todate = datetime(2023, 1, 1)
data = bt.feeds.PandasData(dataname = stock_df, fromdate = fromdate, todate = todate)

#step2: 构建策略
#示例策略：上穿20日线买入，跌穿卖出
class MyStrategy(bt.Strategy):
     # 可变参数
    params = (
        ("maperiod", 20),
    )

    def __init__(self):
        self.order = None
        #indicators:指标计算模块  SimpleMovingAverage:2参数：数据源+周期
        self.ma = bt.indicators.SimpleMovingAverage(self.datas[0], period = self.params.maperiod) #也是拿一条链


    #每个bar都会执行一次next函数，回测的每个日期执行一次，0是回测的当天，-1是前一天，-2是再前一天
    def next(self):
        #判断是否有交易指令正在进行
        if self.order:
            return
        # 判断当前是否有持仓 self.position bool变量
        if(not self.position):
            # 空仓
            #self.datas: 所有的数据，idx=0代表当前数据
            if self.datas[0].close[0] > self.ma[0]:
                self.order = self.buy(size = 200)
        else:
            if self.datas[0].close[0] < self.ma[0]:
                self.order = self.sell(size = 200)


# step3：策略设置
cerebro = bt.Cerebro()    #创建大脑（回测系统）
cerebro.adddata(data)     #添加数据，可以添加多条数据，对应self.datas[n] .close 拿到整个数据线
cerebro.addstrategy(MyStrategy)   #加入自己的策略
startcash = 100000
cerebro.broker.setcash(startcash)   #经纪人 设置初始金额
cerebro.broker.setcommission(0.0002)


#step4: 执行回测
cerebro.run()
endcash = cerebro.broker.getvalue()
print(f"剩余资金{endcash}")












请设置tushare pro的token凭证码，如果没有请访问https://tushare.pro注册申请


Exception: api init error.